# Garbage Collection and Memory Leakage

**Pretty Cool and Interesting stuff ahead**

We have seen about refernce counting in the previous notebook. But, now consider the situation when say an object **My_Var** points to an object **A** which has a parameter/variable which in turn references/points to another object **B**. 

<img src="../files/Circular_Refernce_Normal.png"></img>

Now, when we destroy **my_var**, reference count of object A is zero and hence **A** gets **destroyed**. This in turn causes the reference count of object B to be zero and hence **B** gets **destroyed**. A simple Normal Case

But consider the situation when a parameter/variable of object **B** points to **var_1**.

<img src="../files/Circular_Reference.png"></img>

Now, when we destroy the object **my_var**, reference count of **A** is 1(since var_2 now points to **A**) and referne count of object **B** is 1(since var_1 points to **B**). Therefore, none of object **A** or object **B** gets destroyed. This situation is called as **CIRCULAR REFERENCE**

### EFFECTS

Due to **Circular Reference**, the objects **A** and **B** stays in memory and doesnt get destroted even after the program execution. This leads to **MEMORY  LEAKAGE**. <br>
<br>
<br>
<br>
To overcome this situation, the Python Memory Manager provides the **Garbage Collector**, which erases these kinds of objects.

#### Properties of Garbage Collector(GC)

- By default it is turned on
- Can be controlled programmatically using the python 'gc' module
- runs periodically on its own

### Illustration

In [1]:
#importing librarires
import ctypes
import gc

#function to get the reference count of an object
def ref_count(address):
    return ctypes.c_long.from_address(address).value


#function to check wether the object exists in memory or not
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not found"

In [2]:
class A:
    def __init__(self):
        self.b = B(self)
        print('A: self: {0}, b:{1}'.format(hex(id(self)), hex(id(self.b))))

In [3]:
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))

#### Behind the scenes of the above code snippet
Class A has a parameter 'b'. B pints to memory address of class B. Class B takes class A as argument. Class B has parameter 'a'. Parameter 'a' of class B points to class A

We manually disable the garbage collector for better illustrational purposes

In [4]:
gc.disable()

Let us now createa an instance of A, say **my_var**

In [5]:
my_var = A()

B: self: 0x7efeb4448c10, a: 0x7efeb4448bd0
A: self: 0x7efeb4448bd0, b:0x7efeb4448c10


**NOTE : id(B) = id(b) and id(A) = id(a). This implies the situation of circular reference**

In [7]:
a_id = id(my_var)
b_id = id(my_var.b)

print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 2
refcount(b) = 1
a: Object exists
b: Object exists


Reference Count of a = 2 because both my_var and B point to A<br>
Both a and b exists in memory

In [8]:
my_var= None

We deallocate or remove my_var's reference to create a Circular Reference Situation

In [9]:
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 1
refcount(b) = 1
a: Object exists
b: Object exists


In [10]:
gc.collect()
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 0
refcount(b) = 0
a: Not found
b: Not found


When we run the garbage collector manually using the gc.collect() statement, both object a and b gets destroyed. Hence garbage collector does its job

### EXTRAS

In python versions less than 3.4, the garbage collector(switched on by default) doesnt perform well when a destructor is used in the class. This is because, the destructor will erase a class's objects in a particular order and the Garbage Collector doesnt know that order.<br>
<br>
Whereas this problem is overcomed by python versions 3.4 and above